<a id='section1'></a>
## 1️⃣ הכנה והגדרות ראשוניות

### 📦 התקנת חבילות נדרשות

לפני שמתחילים, וודאו שהתקנתם את החבילות הבאות:

```bash
pip install polars
pip install plotly  # אופציונלי - לויזואליזציה
```

### 💡 מה זה Polars?

**Polars** היא ספריית עיבוד נתונים מהירה במיוחד שנכתבה ב-Rust, עם API נוח ב-Python. היתרונות:

- ⚡ **מהירות** - עד פי 10 מהר יותר מ-Pandas
- 💾 **יעילות זיכרון** - שימוש חכם בזיכרון
- 🔄 **Lazy Evaluation** - אופטימיזציה אוטומטית של שאילתות
- 🎨 **API נקי** - תחביר אינטואיטיבי וברור

### 🔧 ייבוא הספריות

In [ ]:
# ייבוא הספריות הנדרשות
import polars as pl
from polars import selectors as cs

# הצגת גרסת Polars
print(f"גרסת Polars: {pl.__version__}")

### 📊 טעינת הנתונים

נעבוד עם נתונים על סטודנטים בינלאומיים בארה"ב:

In [ ]:
# טעינת הנתונים מקובץ CSV
# הנתיב יכול להשתנות בהתאם למיקום הקובץ שלכם
df = pl.read_csv('../data/academic.csv')

# הצגת 5 השורות הראשונות
df.shape


### 🧹 ניקוי וטיפול ראשוני בנתונים

נבצע מספר שלבי ניקוי ועיבוד ראשוני:

In [ ]:
# שלב 1: שינוי שמות עמודות והמרת טיפוסי נתונים
df = (
    df
    .select(
        pl.col('year').alias('academic_year'),  # שינוי שם העמודה ל-academic_year
        cs.numeric().cast(pl.Int64)  # המרת כל העמודות המספריות ל-Int64
    )
    .filter(
        # סינון רק שנים מ-2018 ואילך
        pl.col('academic_year').str.slice(0, 4).cast(pl.Int32) >= 2018
    )
)

# הצגת התוצאה
df.shape
df.head()

#### 📝 הסבר מפורט:

1. **`pl.col('year').alias('academic_year')`** - בוחרים את עמודת 'year' ומשנים את שמה
2. **`cs.numeric().cast(pl.Int64)`** - בוחרים את כל העמודות המספריות וממירים אותן לטיפוס Int64
3. **`str.slice(0, 4)`** - חותכים את 4 התווים הראשונים מהמחרוזת (למשל "2018" מתוך "2018/19")
4. **`cast(pl.Int32)`** - ממירים את המחרוזת למספר שלם
5. **`>= 2018`** - מסננים רק שורות שהשנה היא 2018 ומעלה

---

<a id='section2'></a>
## 2️⃣ הפיכת עמודות לשורות - Unpivot (Melt)

### 🎯 מתי נשתמש ב-Unpivot?

**Unpivot** (או **Melt**) משמש כאשר רוצים להפוך נתונים מפורמט "רחב" (Wide Format) לפורמט "ארוך" (Long Format).

#### דוגמה:

**לפני (Wide Format):**
```
| year    | students | us_students | undergraduate |
|---------|----------|-------------|---------------|
| 2018/19 | 1095299  | 19828000    | 431930        |
```

**אחרי (Long Format):**
```
| year    | student_type   | count   |
|---------|----------------|--------|
| 2018/19 | students       | 1095299|
| 2018/19 | us_students    | 19828000|
| 2018/19 | undergraduate  | 431930 |
```

### 📊 דוגמה בסיסית

In [ ]:
# הפיכת עמודות לשורות
long_df = df.unpivot(
    index='academic_year',  # עמודה שנשארת קבועה
    on=[
        'students',
        'us_students',
        'undergraduate',
        'graduate',
        'non_degree',
        'opt'
    ],  # עמודות להפוך לשורות
    variable_name='student_type',  # שם העמודה החדשה לסוגי הסטודנטים
    value_name='count'  # שם העמודה החדשה לערכים
)

# הצגת התוצאה
long_df.head()

#### 🔍 ניתוח התוצאה:

בואו נבדוק כמה ערכים ייחודיים יש לנו בעמודת `student_type`:

In [ ]:
# בדיקת הערכים הייחודיים
long_df.select('student_type').unique()

### 🎨 שימוש ב-Selectors

במקום לרשום את כל שמות העמודות באופן ידני, נוכל להשתמש ב-**Selectors** - כלי שמאפשר לבחור עמודות לפי קריטריונים:

In [ ]:
# שימוש ב-Selector לבחירת כל העמודות המספריות
df.unpivot(
    index='academic_year',
    on=cs.numeric()  # בוחר אוטומטית את כל העמודות המספריות
).head()

#### 💡 טיפ חשוב:

> כאשר לא מציינים `variable_name` ו-`value_name`, Polars משתמש בשמות ברירת המחדל: "variable" ו-"value"

### ⚡ שימוש ב-LazyFrame

**LazyFrame** מאפשר ל-Polars לבצע אופטימיזציות על השאילתה לפני שהיא מבוצעת בפועל:

In [ ]:
# המרה ל-LazyFrame
lf = df.lazy()

# ביצוע unpivot על LazyFrame
result = (
    lf
    .unpivot(
        index='academic_year',
        on=cs.numeric(),
        variable_name='student_type',
        value_name='count'
    )
    .collect()  # הרצה בפועל
)

result.head()

#### 🚀 יתרונות LazyFrame:

- ✅ אופטימיזציה אוטומטית של השאילתה
- ✅ ביצוע מהיר יותר על נתונים גדולים
- ✅ שימוש חכם בזיכרון

---

<a id='section3'></a>
## 3️⃣ הפיכת שורות לעמודות - Pivot

### 🎯 מתי נשתמש ב-Pivot?

**Pivot** היא הפעולה ההפוכה ל-Unpivot - הופכת נתונים מפורמט "ארוך" לפורמט "רחב".

#### דוגמה:

**לפני (Long Format):**
```
| year    | student_type | count   |
|---------|--------------|--------|
| 2018/19 | students     | 1095299|
| 2018/19 | graduate     | 377943 |
```

**אחרי (Wide Format):**
```
| year    | students | graduate |
|---------|----------|----------|
| 2018/19 | 1095299  | 377943   |
```

### 🔄 הכנת הנתונים

In [ ]:
# ניצור DataFrame בפורמט Long
long_df = (
    pl.read_csv('../data/academic.csv')
    .select(
        pl.col('year').alias('academic_year'),
        cs.numeric().cast(pl.Int64)
    )
    .filter(
        pl.col('academic_year').str.slice(0, 4).cast(pl.Int32) >= 2018
    )
    .unpivot(
        index='academic_year',
        on=[
            'students',
            'us_students',
            'undergraduate',
            'graduate',
            'non_degree',
            'opt'
        ],
        variable_name='student_type',
        value_name='count'
    )
)

# הצגת הנתונים
long_df.head()

### 📊 ביצוע Pivot בסיסי

In [ ]:
# הפיכת שורות לעמודות
wide_df = (
    long_df
    .pivot(
        index='academic_year',  # עמודת המפתח
        values='count',  # הערכים למלא בטבלה
        on='student_type'  # העמודה שתהפוך לעמודות מרובות
    )
)

wide_df

### 🔧 אלטרנטיבה עם Group By

ניתן גם להשיג תוצאה דומה באמצעות `group_by` ו-`agg`:

In [ ]:
# שיטה אלטרנטיבית עם group_by
(
    long_df
    .group_by('academic_year', maintain_order=True)
    .agg(
        pl.col('count').filter(pl.col('student_type') == 'students').sum().alias('students'),
        pl.col('count').filter(pl.col('student_type') == 'us_students').sum().alias('us_students'),
        pl.col('count').filter(pl.col('student_type') == 'undergraduate').sum().alias('undergraduate'),
        pl.col('count').filter(pl.col('student_type') == 'graduate').sum().alias('graduate'),
        pl.col('count').filter(pl.col('student_type') == 'non_degree').sum().alias('non_degree'),
        pl.col('count').filter(pl.col('student_type') == 'opt').sum().alias('opt')
    )
)

### 🎨 גרסה דינמית יותר

במקום לכתוב כל עמודה באופן ידני, נוכל ליצור את הקוד באופן דינמי:

In [ ]:
# יצירת רשימת סוגי הסטודנטים
student_types = [
    col for col in long_df.select('student_type').unique().to_series().to_list()
]

# יצירת רשימת אגרגציות באופן דינמי
agg_cols = [
    (
        pl.col('count')
        .filter(pl.col('student_type') == stu_type)
        .sum()
        .alias(stu_type)
    )
    for stu_type in student_types
]

# ביצוע הקיבוץ
(
    long_df
    .group_by('academic_year', maintain_order=True)
    .agg(agg_cols)
)

### ⚠️ טיפול בכפילויות

מה קורה כשיש כפילויות בנתונים?

#### בעיה: נתונים עם כפילויות

In [ ]:
# יצירת DataFrame עם כפילויות
wide_df_with_dups = (
    pl.concat([
        df.head(1),  # שורה ראשונה
        df  # כל הנתונים
    ])
)

wide_df_with_dups

In [ ]:
# הפיכה ל-Long Format
long_df_with_dups = (
    wide_df_with_dups
    .unpivot(
        index='academic_year',
        on=[
            'students',
            'us_students',
            'undergraduate',
            'graduate',
            'non_degree',
            'opt'
        ],
        variable_name='student_type',
        value_name='count'
    )
)

long_df_with_dups.head(10)

#### ❌ מה קורה אם נעשה Pivot?

In [ ]:
# זה יגרום לשגיאה!
try:
    (
        long_df_with_dups
        .pivot(
            index='academic_year',
            values='count',
            on='student_type'
        )
    )
except Exception as e:
    print(f"❌ שגיאה: {e}")

#### ✅ פתרון: שימוש בפונקציית אגרגציה

In [ ]:
# פתרון 1: שימוש במינימום
(
    long_df_with_dups
    .pivot(
        index='academic_year',
        values='count',
        on='student_type',
        aggregate_function='min'  # בוחר את הערך המינימלי
    )
)

In [ ]:
# פתרון 2: שמירת כל הערכים כרשימה
(
    long_df_with_dups
    .pivot(
        index='academic_year',
        values='count',
        on='student_type',
        aggregate_function=pl.element()  # יוצר רשימה של כל הערכים
    )
)

#### 📚 אפשרויות לפונקציות אגרגציה:

- `'min'` - ערך מינימלי
- `'max'` - ערך מקסימלי
- `'sum'` - סכום
- `'mean'` - ממוצע
- `'count'` - ספירה
- `pl.element()` - רשימה של כל הערכים

---

<a id='section4'></a>
## 4️⃣ חיבור טבלאות - Join

### 🎯 מה זה Join?

**Join** מאפשר לנו לחבר שתי טבלאות על בסיס עמודה משותפת (מפתח).

### 🔧 הכנת הנתונים

In [ ]:
# טבלה 1: נתוני סטודנטים
academic_df = (
    pl.read_csv('../data/academic.csv')
    .select(
        pl.col('year').alias('academic_year'),
        cs.numeric().cast(pl.Int64)
    )
    .filter(pl.col('academic_year').str.slice(0, 4).cast(pl.Int32) >= 2018)
)

print("טבלה 1: נתוני סטודנטים")
academic_df

In [ ]:
# טבלה 2: נתוני סטטוס
status_df = (
    pl.read_csv('../data/status.csv')
    .with_columns(
        cs.float().cast(pl.Int64)
    )
)

print("טבלה 2: נתוני סטטוס")
status_df

### 🔗 ביצוע Join בסיסי

In [ ]:
# חיבור שתי הטבלאות
joined_df = (
    academic_df
    .join(
        status_df,
        left_on='academic_year',  # עמודת המפתח בטבלה השמאלית
        right_on='year',  # עמודת המפתח בטבלה הימנית
        how='inner'  # סוג ה-join
    )
    .select(
        'academic_year',
        'students',
        cs.contains('visa')  # בחירת כל עמודות ה-visa
    )
)

joined_df.head()

### 📊 סוגי Join

| סוג Join | תיאור | מתי להשתמש |
|----------|--------|------------|
| **inner** | רק שורות עם התאמה בשתי הטבלאות | כאשר רוצים רק נתונים משותפים |
| **left** | כל השורות מהטבלה השמאלית | כאשר רוצים לשמור את כל הנתונים מהטבלה הראשונה |
| **right** | כל השורות מהטבלה הימנית | כאשר רוצים לשמור את כל הנתונים מהטבלה השנייה |
| **outer** | כל השורות משתי הטבלאות | כאשר רוצים את כל הנתונים |

### 🎨 ויזואליזציה של הנתונים

In [ ]:
# יצירת ויזואליזציה
import plotly.express as px

# הכנת הנתונים לגרף
viz_df = (
    joined_df
    .unpivot(
        index=['academic_year', 'students'],
        on=cs.contains('visa'),
        variable_name='visa_type',
        value_name='count'
    )
    .with_columns(
        (pl.col('count') / pl.col('students')).alias('percent_of_total')
    )
)

# יצירת גרף עמודות מוערם
fig = px.bar(
    x=viz_df['academic_year'],
    y=viz_df['percent_of_total'],
    color=viz_df['visa_type'],
    barmode='stack',
    text_auto='.1%',
    title='התפלגות סטודנטים לפי סוג ויזה',
    labels={
        'x': 'שנה אקדמית',
        'y': 'אחוז מסך הסטודנטים'
    },
)

fig.update_layout(
    autosize=True,
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    yaxis_tickformat='0%'
)

fig.update_traces(textposition='inside')
fig.show()

### 🔍 וולידציה של Join

ניתן לבדוק שהחיבור נעשה כראוי באמצעות פרמטר `validate`:

In [ ]:
# הכנת long_df
status_long_df = (
    status_df
    .unpivot(
        index='year',
        on=cs.contains('visa'),
        variable_name='visa_type',
        value_name='count'
    )
)

status_long_df.head()

In [ ]:
# Join עם וולידציה - 1:m (one-to-many)
(
    academic_df
    .join(
        status_long_df,
        left_on='academic_year',
        right_on='year',
        how='inner',
        validate='1:m'  # מבטיח שלכל שורה בטבלה השמאלית יש מספר שורות בימנית
    )
    .select(
        'academic_year',
        'students',
        'visa_type',
        'count'
    )
)

#### ❌ דוגמה לשגיאת וולידציה

In [ ]:
# זה ייכשל כי יש מספר שורות לכל academic_year
try:
    (
        academic_df
        .join(
            status_long_df,
            left_on='academic_year',
            right_on='year',
            how='inner',
            validate='1:1'  # דורש יחס 1:1 - זה ייכשל!
        )
    )
except Exception as e:
    print(f"❌ שגיאה כצפוי: {e}")

### ⚡ Join מסוג ASOF

**ASOF Join** משמש לחיבור נתונים על בסיס הערך הקרוב ביותר (למשל, תאריכים):

In [ ]:
# דוגמה ל-ASOF Join
a = pl.DataFrame({
    'int': [1, 2, 3],
    'value': [10, 20, 30]
}).set_sorted('int')

b = pl.DataFrame({
    'int': [4, 5, 6]
}).set_sorted('int')

# חיפוש הערך הקרוב ביותר אחורה
b.join_asof(a, on='int', strategy='backward')

---

<a id='section5'></a>
## 5️⃣ שרשור טבלאות - Concatenation

### 🎯 מה זה Concatenation?

**Concatenation** מאפשר לחבר מספר טבלאות זו לזו - אנכית (שורות) או אופקית (עמודות).

### 📊 הכנת נתונים לדוגמה

In [ ]:
# חלוקת הנתונים ל-3 חלקים
df1 = academic_df.head(2)
df2 = academic_df.slice(2, 2)
df3 = academic_df.tail(1)

print("DataFrame 1:")
display(df1)

print("\nDataFrame 2:")
display(df2)

print("\nDataFrame 3:")
display(df3)

### ⬇️ שרשור אנכי (Vertical)

הוספת שורות זו מתחת לזו:

In [ ]:
# שיטה 1: שימוש ב-concat
pl.concat(
    [df1, df2, df3],
    how='vertical'
)

In [ ]:
# שיטה 2: שימוש ב-vstack
df1.vstack(df2).vstack(df3)

#### 💡 הבדל בין concat ל-vstack:

- **`concat`** - גמיש יותר, מאפשר שרשור של מספר DataFrames בבת אחת
- **`vstack`** - מהיר יותר, אבל עובד רק עם שני DataFrames בכל פעם

### ➡️ שרשור אופקי (Horizontal)

הוספת עמודות זו ליד זו:

In [ ]:
# שיטה 1: שימוש ב-concat
pl.concat(
    [
        df1.select('academic_year'),
        df2.select('students')
    ],
    how='horizontal'
)

#### ⚠️ אזהרה - אורכים שונים!

In [ ]:
# מה קורה כשהאורכים שונים?
pl.concat(
    [
        df1.select('academic_year'),  # 2 שורות
        df2.select('students'),  # 2 שורות
        df3.select('us_students')  # 1 שורה
    ],
    how='horizontal'
)

> 📝 **שימו לב:** כאשר האורכים שונים, Polars ימלא את החסר ב-null

### 🔧 שימוש ב-hstack

In [ ]:
# שיטה 2: שימוש ב-hstack
(
    df1
    .select('academic_year')
    .hstack(df2.select('students'))
)

#### ❌ hstack דורש אורכים זהים!

In [ ]:
# זה ייכשל!
try:
    (
        df1
        .select('academic_year')
        .hstack(df2.select('students'))
        .hstack(df3.select('us_students'))  # אורך שונה!
    )
except Exception as e:
    print(f"❌ שגיאה: {e}")

### 🔄 extend - שינוי במקום

**extend** דומה ל-vstack, אבל משנה את ה-DataFrame המקורי:

In [ ]:
# יצירת עותק של df1
df1_copy = df1.clone()

# הוספת שורות במקום
df1_copy.extend(df2).extend(df3)

#### ⚠️ שימו לב:

> **extend** משנה את ה-DataFrame המקורי, בעוד ש-**vstack** יוצר DataFrame חדש

---

<a id='section6'></a>
## 6️⃣ טכניקות עיצוב נוספות

### 🔀 Partition - חלוקה לחלקים

**partition_by** מחלקת את ה-DataFrame למספר DataFrames קטנים לפי ערך של עמודה:

In [ ]:
# חלוקה לפי שנה אקדמית
partitions = academic_df.partition_by('academic_year')

# הצגת החלוקה הראשונה
print(f"מספר חלוקות: {len(partitions)}")
print("\nחלוקה ראשונה:")
partitions[0]

### 🔄 Transpose - היפוך שורות ועמודות

**transpose** הופך את השורות לעמודות והעמודות לשורות:

In [ ]:
# Transpose בסיסי
academic_df.transpose(include_header=True)

### 📐 Reshape - שינוי צורה

**reshape** מאפשר לשנות את המימדים של העמודות:

In [ ]:
# שינוי צורה לעמודה של מערכים
(
    academic_df
    .select(
        pl.col('academic_year', 'students').reshape((1, 5))
    )
)

---

<a id='section7'></a>
## 7️⃣ תרגילים מעשיים

### תרגיל 1: Unpivot בסיסי ⭐

**משימה:** צרו DataFrame בפורמט Long שמכיל רק את עמודות הסטודנטים הזרים.

In [ ]:
# התחילו כאן!
# רמז: השתמשו ב-unpivot עם העמודות: undergraduate, graduate, non_degree


<details>
<summary>💡 לחצו כאן לפתרון</summary>

```python
result = df.unpivot(
    index='academic_year',
    on=['undergraduate', 'graduate', 'non_degree'],
    variable_name='student_category',
    value_name='count'
)
result
```
</details>

---

### תרגיל 2: Pivot עם אגרגציה ⭐⭐

**משימה:** קחו את `status_long_df` והפכו אותו חזרה לפורמט Wide, תוך שימוש במקסימום כפונקציית אגרגציה.

In [ ]:
# התחילו כאן!


<details>
<summary>💡 לחצו כאן לפתרון</summary>

```python
result = (
    status_long_df
    .pivot(
        index='year',
        values='count',
        on='visa_type',
        aggregate_function='max'
    )
)
result
```
</details>

---

### תרגיל 3: Join מורכב ⭐⭐⭐

**משימה:** צרו DataFrame שמשלב את `academic_df` עם `status_df`, ומחשב את האחוז של כל סוג ויזה מתוך סך הסטודנטים הבינלאומיים.

In [ ]:
# התחילו כאן!


<details>
<summary>💡 לחצו כאן לפתרון</summary>

```python
result = (
    academic_df
    .join(
        status_df,
        left_on='academic_year',
        right_on='year',
        how='inner'
    )
    .select(
        'academic_year',
        'students',
        cs.contains('visa')
    )
    .unpivot(
        index=['academic_year', 'students'],
        on=cs.contains('visa'),
        variable_name='visa_type',
        value_name='count'
    )
    .with_columns(
        ((pl.col('count') / pl.col('students')) * 100).alias('percentage')
    )
)
result
```
</details>

---

### תרגיל 4: אתגר מתקדם 🏆

**משימה:** צרו ניתוח שמשווה את השינוי באחוז הסטודנטים מכל סוג ויזה בין 2018 ל-2022.

In [ ]:
# התחילו כאן!


---

<a id='section8'></a>
## 8️⃣ סיכום וטיפים

### 📚 מה למדנו?

בסיום מדריך זה, למדתם:

1. ✅ **Unpivot** - הפיכת עמודות לשורות לניתוח טוב יותר
2. ✅ **Pivot** - הפיכת שורות לעמודות לתצוגה נוחה
3. ✅ **Join** - חיבור טבלאות על בסיס מפתחות משותפים
4. ✅ **Concatenation** - שרשור טבלאות אנכית ואופקית
5. ✅ **Partition, Transpose, Reshape** - טכניקות נוספות לעיצוב נתונים

### 💡 טיפים חשובים

#### 🎯 מתי להשתמש ב-Unpivot?

- כאשר יש לכם עמודות רבות של אותו סוג מדידה
- כאשר רוצים לעשות ניתוח או ויזואליזציה לפי קטגוריה
- כאשר רוצים לקבץ נתונים לפי משתנה קטגורי

#### 🎯 מתי להשתמש ב-Pivot?

- כאשר רוצים לראות השוואה בין קטגוריות
- כאשר רוצים ליצור טבלה מסודרת לדוח
- כאשר רוצים לחשב סטטיסטיקות צולבות

#### 🎯 מתי להשתמש ב-Join?

- כאשר יש לכם מידע משלים בטבלאות שונות
- כאשר רוצים להעשיר נתונים ממקורות שונים
- כאשר יש קשרים בין טבלאות שונות

### ⚡ ביצועים

#### עצות לביצועים טובים:

1. **השתמשו ב-LazyFrame** כאשר האפשר - זה מאפשר אופטימיזציות
2. **סננו נתונים מוקדם** - לפני unpivot או join
3. **השתמשו ב-Selectors** במקום רישום ידני של עמודות
4. **בדקו את גודל הנתונים** לפני פעולות כבדות

### 🔍 טיפים לדיבאג

1. **בדקו את המבנה** - השתמשו ב-`df.head()` ו-`df.describe()` לפני ואחרי כל פעולה
2. **ספרו שורות** - וודאו שמספר השורות הגיוני אחרי join או concat
3. **בדקו null values** - אחרי join, בדקו אם יש ערכים חסרים
4. **השתמשו ב-validate** - ב-join כדי לוודא שהקשר נכון

### 📊 משאבים נוספים

- 📖 [תיעוד Polars הרשמי](https://pola-rs.github.io/polars/)
- 💬 [קהילת Polars ב-Discord](https://discord.gg/4UfP5cfBE7)
- 📺 [הרצאות על Polars ב-YouTube](https://www.youtube.com/results?search_query=polars+python)
- 📝 [Polars Cookbook](https://github.com/pola-rs/polars/tree/main/examples)

### 🎓 המשך לימוד

נושאים מומלצים להמשך:

1. **Window Functions** - פונקציות חלון מתקדמות
2. **Lazy API** - אופטימיזציה מתקדמת
3. **Streaming** - עבודה עם נתונים גדולים מאוד
4. **User Defined Functions** - פונקציות מותאמות אישית

---

## 🎉 סיום

**כל הכבוד!** סיימתם את המדריך המקיף לעיצוב וארגון נתונים ב-Polars.

זכרו - התרגול הוא המפתח להצלחה! נסו ליישם את מה שלמדתם על נתונים אמיתיים שלכם.

**בהצלחה! 🚀**

---

*מדריך זה נוצר במיוחד עבורכם בעברית. לשאלות והערות, אנא פנו למקורות הנוספים.*